<a href="https://colab.research.google.com/github/oyjuffer/DL-Final/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from keras.callbacks import EarlyStopping

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

random_seed = 40

# File loading

In [101]:
data_dir= "emotion-detection-fer/"

# Data loading
Load train and validation sets.Images are resized to 96x96 pixels, as this is the smallest size available for the MobileNetV2 model. Even though the images are grayscale, they are loaded as RGB as mobilenet requires 3 channels.

In [ ]:
image_size = 96 # Images are 48, but smallest model is 96
batch_size = 512 # default batch size

ext_flag=0

if ext_flag:

  train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir + "train_ext/",
    image_size=(image_size, image_size),
    batch_size=batch_size,
    color_mode='rgb')
  
  val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir + "val_ext/",
    image_size=(image_size, image_size),
    batch_size=batch_size,
    color_mode='rgb')
  
else:
  
  train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir + "train/",
    image_size=(image_size, image_size),
    batch_size=batch_size,
    color_mode='rgb')

  train_ds, val_ds = tf.keras.utils.split_dataset(train_ds, left_size=0.8, seed=random_seed)

test_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir + "test/",
  image_size=(image_size, image_size),
  batch_size=batch_size,
  color_mode='rgb')

### List the different classes

In [ ]:
class_names = test_ds.class_names
print(class_names)

### Data preview

In [ ]:
# Optional, just to get an understanding of whats happening
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in val_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

# Transfer Learning
First, the input is preprocessed to convert the values in the images, which are between [0, 255], to values between [-1, 1]. This is done to match the expected input of the MobileNetV2 model.

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

Load the MobileNetV2 model, with the weights pre-trained on ImageNet. The model is set to not trainable, as we only want to use the convolutional layers for feature extraction.

In [ ]:
IMG_SHAPE = (image_size, image_size, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = False

## Show model architecture

In [ ]:
base_model.summary()

## Add classification layer

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
num_classes = len(class_names)
prediction_layer = tf.keras.layers.Dense(num_classes)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

### Build model

In [ ]:
inputs = tf.keras.Input(shape=(image_size, image_size, 3))
x = inputs
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

### Compile model

In [ ]:
base_learning_rate = 0.0001

model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

### Show model architecture again

In [ ]:
# Optional, just to get an understanding of whats happening
model.summary()

# Train the model

In [ ]:
def train_model(model, train=train_ds, validation=val_ds, epochs=10, patience=4, workers=4):
  early_stop = EarlyStopping(monitor="val_loss", patience=patience)
  
  callbacks = [early_stop]

  history = model.fit(
    train_ds,
    validation_data=val_ds,
    callbacks=callbacks,
    epochs=epochs,
    workers=workers
  )
  
  return history

In [ ]:
history = train_model(model, epochs=200, patience=25)

In [103]:
test_loss, test_acc = model.evaluate(test_ds)

15/15 [==============================] - 19s 1s/step - loss: 3.6122 - accuracy: 0.6017


In [ ]:
model.save("cnn_trained_transfer", overwrite=True)

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

test_loss, test_acc = model.evaluate(test_ds)

labels = []; predictions = []
for element in test_ds.as_numpy_iterator():
    preds = model.predict(element[0], batch_size=batch_size, verbose=0)
    
    for ii in range(len(preds)):
        labels.append(class_names[element[1][ii]])
        predictions.append(class_names[preds[ii].argmax()])
        
cm = confusion_matrix(labels, predictions, labels=class_names, normalize='pred')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot()

plt.savefig('cnn_confussion_matrix.png')
plt.show()

## Learning curves

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
# plt.ylim([min(plt.ylim()),0.6])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()